## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Team
- Loading the specification and running the resulting agent

 

In [1]:
# import logging
# from autogen_agentchat.logging import ConsoleLogHandler
# from autogen_agentchat import EVENT_LOGGER_NAME
# logger = logging.getLogger(EVENT_LOGGER_NAME)
# logger.handlers = [ConsoleLogHandler()]
# logger.setLevel(logging.INFO)


In [2]:
from autogenstudio.database import AgentFactory 
from autogenstudio.teammanager import TeamManager
import json 


team_json_spec = json.load(open("team.json")) 
wm = TeamManager() 

print(team_json_spec)

{'name': 'weather_team', 'participants': [{'name': 'writing_agent', 'model_client': {'model': 'gpt-4o-2024-08-06', 'model_type': 'OpenAIChatCompletionClient'}, 'tools': [{'name': 'get_weather', 'description': 'Get the weather for a city', 'content': 'async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."'}], 'agent_type': 'AssistantAgent'}], 'termination_condition': {'termination_type': 'MaxMessageTermination', 'max_messages': 5}, 'team_type': 'RoundRobinGroupChat', 'model_client': None}


In [3]:
result = await wm.run(task="What is the weather in New York?", team_config=team_json_spec)

*** team config *** name='weather_team' participants=[AgentConfig(name='writing_agent', agent_type=<AgentTypes.assistant: 'AssistantAgent'>, system_message=None, model_client=ModelConfig(model='gpt-4o-2024-08-06', model_type=<ModelTypes.openai: 'OpenAIChatCompletionClient'>, api_key=None, base_url=None), tools=[ToolConfig(name='get_weather', description='Get the weather for a city', content='async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."')], description=None)] team_type=<TeamTypes.round_robin: 'RoundRobinGroupChat'> model_client=None termination_condition=TerminationConfig(termination_type=<TerminationTypes.max_messages: 'MaxMessageTermination'>, max_messages=5, text=None)


In [4]:
stream =  wm.run_stream(task="What is the weather in New York?", team_config=team_json_spec) 
async for response in stream:
    print(type(response), response)

*** team config *** name='weather_team' participants=[AgentConfig(name='writing_agent', agent_type=<AgentTypes.assistant: 'AssistantAgent'>, system_message=None, model_client=ModelConfig(model='gpt-4o-2024-08-06', model_type=<ModelTypes.openai: 'OpenAIChatCompletionClient'>, api_key=None, base_url=None), tools=[ToolConfig(name='get_weather', description='Get the weather for a city', content='async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."')], description=None)] team_type=<TeamTypes.round_robin: 'RoundRobinGroupChat'> model_client=None termination_condition=TerminationConfig(termination_type=<TerminationTypes.max_messages: 'MaxMessageTermination'>, max_messages=5, text=None)
<class 'autogen_agentchat.messages.TextMessage'> source='user' models_usage=None content='What is the weather in New York?'
<class 'autogen_agentchat.messages.ToolCallMessage'> source='writing_agent' models_usage=RequestUsage(prompt_tokens=86, completion_tokens=

## Testing DB API

In [5]:
from autogenstudio.database import DatabaseManager 

# create a database
dbmanager = DatabaseManager(engine_uri="sqlite:///test.db")
dbmanager.create_db_and_tables() 

2024-11-06 13:53:24.900 | INFO     | autogenstudio.database.schema_manager:_ensure_alembic_setup:280 - Force initialization requested. Cleaning up existing configuration...
2024-11-06 13:53:24.900 | INFO     | autogenstudio.database.schema_manager:_cleanup_existing_alembic:51 - Cleaning up existing Alembic configuration while preserving versions...
2024-11-06 13:53:24.901 | INFO     | autogenstudio.database.schema_manager:_cleanup_existing_alembic:56 - Preserving existing versions directory
2024-11-06 13:53:24.902 | INFO     | autogenstudio.database.schema_manager:_cleanup_existing_alembic:68 - Removed file: /Users/victordibia/projects/hax/autogen/python/packages/autogen-studio/autogenstudio/database/alembic/env.py
2024-11-06 13:53:24.903 | INFO     | autogenstudio.database.schema_manager:_cleanup_existing_alembic:76 - Removed existing alembic.ini: /Users/victordibia/projects/hax/autogen/python/packages/autogen-studio/autogenstudio/database/alembic.ini
2024-11-06 13:53:24.903 | INFO   

In [6]:

from sqlmodel import Session, text, select
from autogenstudio.datamodel import Model, ModelConfig, ModelTypes, Team, TeamConfig, TeamTypes, Agent, AgentConfig, AgentTypes, Tool, ToolConfig, LinkTypes

user_id = "guestuser@gmail.com"
from autogenstudio.datamodel import ModelConfig, Model, TeamConfig, Team, Tool, Agent, AgentConfig, TerminationConfig, TerminationTypes, ModelTypes, TeamTypes, AgentTypes, ToolConfig, LinkTypes

gpt4_model = Model(user_id=user_id, config= ModelConfig(model="gpt-4o-2024-08-06", model_type=ModelTypes.openai).model_dump() )

weather_tool = Tool(user_id=user_id, config=ToolConfig(name="get_weather", description="Get the weather for a city", content="async def get_weather(city: str) -> str:\n    return f\"The weather in {city} is 73 degrees and Sunny.\"").model_dump() )

adding_tool = Tool(user_id=user_id, config=ToolConfig(name="add", description="Add two numbers", content="async def add(a: int, b: int) -> int:\n    return a + b").model_dump() )

writing_agent = Agent(user_id=user_id, 
                      config=AgentConfig(
                          name="writing_agent", 
                          tools=[weather_tool.config], 
                          agent_type=AgentTypes.assistant,
                          model_client=gpt4_model.config
                          ).model_dump()
                    )

team = Team(user_id=user_id, config=TeamConfig(
    name="weather_team",
    participants=[writing_agent.config],
    termination_condition=TerminationConfig(termination_type=TerminationTypes.max_messages, max_messages=5).model_dump(),
    team_type=TeamTypes.round_robin
    ).model_dump()
)

with Session(dbmanager.engine) as session:
    session.add(gpt4_model)
    session.add(weather_tool)
    session.add(adding_tool)
    session.add(writing_agent)
    session.add(team)
    session.commit()

    dbmanager.link(LinkTypes.AGENT_MODEL, writing_agent.id, gpt4_model.id)
    dbmanager.link(LinkTypes.AGENT_TOOL, writing_agent.id, weather_tool.id)
    dbmanager.link(LinkTypes.AGENT_TOOL, writing_agent.id, adding_tool.id)
    dbmanager.link(LinkTypes.TEAM_AGENT, team.id, writing_agent.id)


In [7]:
from autogenstudio.database import ConfigurationManager 
config_manager = ConfigurationManager(dbmanager)

In [8]:
result = config_manager.import_config("./team.json", user_id=user_id)

*** team config *** name='weather_team' participants=[AgentConfig(name='writing_agent', agent_type=<AgentTypes.assistant: 'AssistantAgent'>, system_message=None, model_client=ModelConfig(model='gpt-4o-2024-08-06', model_type=<ModelTypes.openai: 'OpenAIChatCompletionClient'>, api_key=None, base_url=None), tools=[ToolConfig(name='get_weather', description='Get the weather for a city', content='async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."')], description=None)] team_type=<TeamTypes.round_robin: 'RoundRobinGroupChat'> model_client=None termination_condition=TerminationConfig(termination_type=<TerminationTypes.max_messages: 'MaxMessageTermination'>, max_messages=5, text=None)


In [9]:
result

Response(message='Team Created Successfully', status=True, data={'created_at': datetime.datetime(2024, 11, 6, 13, 53, 25, 101647), 'config': {'name': 'weather_team', 'participants': [{'name': 'writing_agent', 'agent_type': 'AssistantAgent', 'system_message': None, 'model_client': {'model': 'gpt-4o-2024-08-06', 'model_type': 'OpenAIChatCompletionClient', 'api_key': None, 'base_url': None}, 'tools': [{'name': 'get_weather', 'description': 'Get the weather for a city', 'content': 'async def get_weather(city: str) -> str:\n    return f"The weather in {city} is 73 degrees and Sunny."'}], 'description': None}], 'team_type': 'RoundRobinGroupChat', 'model_client': None, 'termination_condition': {'termination_type': 'MaxMessageTermination', 'max_messages': 5, 'text': None}}, 'user_id': 'guestuser@gmail.com', 'id': 9, 'updated_at': datetime.datetime(2024, 11, 6, 13, 53, 25, 101651), 'version': '0.0.1'})